In [2]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
from PIL import Image, ImageDraw, ImageFont
from IPython.display import display
import glob
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [3, 1]

Instructions for updating:
non-resource variables are not supported in the long term


# 4. Model Hyperparameters

In [3]:
_BATCH_NORM_DECAY = 0.9
_BATCH_NORM_EPSILON = 1e-05
_LEAKY_RELU = 0.1
_ANCHORS = [(10, 13), (16, 30), (33, 23),
            (30, 61), (62, 45), (59, 119),
            (116, 90), (156, 198), (373, 326)]
_MODEL_SIZE = (416, 416)

# 5. Batch norm and fixed padding

It's useful to define batch_norm function since the model uses batch norms with shared parameters heavily. Also, same as ResNet, Yolo uses convolution with fixed padding, which means that padding is defined only by the size of the kernel.

In [4]:
def batch_norm(inputs, training, data_format):
    """Performs a batch normalization using a standard set of parameters."""
    return tf.layers.batch_normalization(
        inputs=inputs, axis=1 if data_format == 'channels_first' else 3,
        momentum=_BATCH_NORM_DECAY, epsilon=_BATCH_NORM_EPSILON,
        scale=True, training=training)


def fixed_padding(inputs, kernel_size, data_format):
    """ResNet implementation of fixed padding.

    Pads the input along the spatial dimensions independently of input size.

    Args:
        inputs: Tensor input to be padded.
        kernel_size: The kernel to be used in the conv2d or max_pool2d.
        data_format: The input format.
    Returns:
        A tensor with the same format as the input.
    """
    pad_total = kernel_size - 1
    pad_beg = pad_total // 2
    pad_end = pad_total - pad_beg

    if data_format == 'channels_first':
        padded_inputs = tf.pad(inputs, [[0, 0], [0, 0],
                                        [pad_beg, pad_end],
                                        [pad_beg, pad_end]])
    else:
        padded_inputs = tf.pad(inputs, [[0, 0], [pad_beg, pad_end],
                                        [pad_beg, pad_end], [0, 0]])
    return padded_inputs


def conv2d_fixed_padding(inputs, filters, kernel_size, data_format, strides=1):
    """Strided 2-D convolution with explicit padding."""
    if strides > 1:
        inputs = fixed_padding(inputs, kernel_size, data_format)

    return tf.layers.conv2d(
        inputs=inputs, filters=filters, kernel_size=kernel_size,
        strides=strides, padding=('SAME' if strides == 1 else 'VALID'),
        use_bias=False, data_format=data_format)

# 6. Feature extraction: Darknet-53

For feature extraction, Yolo uses Darknet-53 neural net pre-trained on ImageNet. Same as ResNet, Darknet-53 has shortcut (residual) connections, which help information from earlier layers flow further. We omit the last 3 layers (Avgpool, Connected, and Softmax) since we only need the features.

In [5]:
def darknet53_residual_block(inputs, filters, training, data_format,
                             strides=1):
    """Creates a residual block for Darknet."""
    shortcut = inputs

    inputs = conv2d_fixed_padding(
        inputs, filters=filters, kernel_size=1, strides=strides,
        data_format=data_format)
    inputs = batch_norm(inputs, training=training, data_format=data_format)
    inputs = tf.nn.leaky_relu(inputs, alpha=_LEAKY_RELU)

    inputs = conv2d_fixed_padding(
        inputs, filters=2 * filters, kernel_size=3, strides=strides,
        data_format=data_format)
    inputs = batch_norm(inputs, training=training, data_format=data_format)
    inputs = tf.nn.leaky_relu(inputs, alpha=_LEAKY_RELU)

    inputs += shortcut

    return inputs


def darknet53(inputs, training, data_format):
    """Creates Darknet53 model for feature extraction."""
    inputs = conv2d_fixed_padding(inputs, filters=32, kernel_size=3,
                                  data_format=data_format)
    inputs = batch_norm(inputs, training=training, data_format=data_format)
    inputs = tf.nn.leaky_relu(inputs, alpha=_LEAKY_RELU)
    inputs = conv2d_fixed_padding(inputs, filters=64, kernel_size=3,
                                  strides=2, data_format=data_format)
    inputs = batch_norm(inputs, training=training, data_format=data_format)
    inputs = tf.nn.leaky_relu(inputs, alpha=_LEAKY_RELU)

    inputs = darknet53_residual_block(inputs, filters=32, training=training,
                                      data_format=data_format)

    inputs = conv2d_fixed_padding(inputs, filters=128, kernel_size=3,
                                  strides=2, data_format=data_format)
    inputs = batch_norm(inputs, training=training, data_format=data_format)
    inputs = tf.nn.leaky_relu(inputs, alpha=_LEAKY_RELU)

    for _ in range(2):
        inputs = darknet53_residual_block(inputs, filters=64,
                                          training=training,
                                          data_format=data_format)

    inputs = conv2d_fixed_padding(inputs, filters=256, kernel_size=3,
                                  strides=2, data_format=data_format)
    inputs = batch_norm(inputs, training=training, data_format=data_format)
    inputs = tf.nn.leaky_relu(inputs, alpha=_LEAKY_RELU)

    for _ in range(8):
        inputs = darknet53_residual_block(inputs, filters=128,
                                          training=training,
                                          data_format=data_format)

    route1 = inputs

    inputs = conv2d_fixed_padding(inputs, filters=512, kernel_size=3,
                                  strides=2, data_format=data_format)
    inputs = batch_norm(inputs, training=training, data_format=data_format)
    inputs = tf.nn.leaky_relu(inputs, alpha=_LEAKY_RELU)

    for _ in range(8):
        inputs = darknet53_residual_block(inputs, filters=256,
                                          training=training,
                                          data_format=data_format)

    route2 = inputs

    inputs = conv2d_fixed_padding(inputs, filters=1024, kernel_size=3,
                                  strides=2, data_format=data_format)
    inputs = batch_norm(inputs, training=training, data_format=data_format)
    inputs = tf.nn.leaky_relu(inputs, alpha=_LEAKY_RELU)

    for _ in range(4):
        inputs = darknet53_residual_block(inputs, filters=512,
                                          training=training,
                                          data_format=data_format)

    return route1, route2, inputs

# 7. Convolution layers

Yolo has a large number of convolutional layers. It's useful to group them in blocks.

In [6]:
def yolo_convolution_block(inputs, filters, training, data_format):
    """Creates convolution operations layer used after Darknet."""
    inputs = conv2d_fixed_padding(inputs, filters=filters, kernel_size=1,
                                  data_format=data_format)
    inputs = batch_norm(inputs, training=training, data_format=data_format)
    inputs = tf.nn.leaky_relu(inputs, alpha=_LEAKY_RELU)

    inputs = conv2d_fixed_padding(inputs, filters=2 * filters, kernel_size=3,
                                  data_format=data_format)
    inputs = batch_norm(inputs, training=training, data_format=data_format)
    inputs = tf.nn.leaky_relu(inputs, alpha=_LEAKY_RELU)

    inputs = conv2d_fixed_padding(inputs, filters=filters, kernel_size=1,
                                  data_format=data_format)
    inputs = batch_norm(inputs, training=training, data_format=data_format)
    inputs = tf.nn.leaky_relu(inputs, alpha=_LEAKY_RELU)

    inputs = conv2d_fixed_padding(inputs, filters=2 * filters, kernel_size=3,
                                  data_format=data_format)
    inputs = batch_norm(inputs, training=training, data_format=data_format)
    inputs = tf.nn.leaky_relu(inputs, alpha=_LEAKY_RELU)

    inputs = conv2d_fixed_padding(inputs, filters=filters, kernel_size=1,
                                  data_format=data_format)
    inputs = batch_norm(inputs, training=training, data_format=data_format)
    inputs = tf.nn.leaky_relu(inputs, alpha=_LEAKY_RELU)

    route = inputs

    inputs = conv2d_fixed_padding(inputs, filters=2 * filters, kernel_size=3,
                                  data_format=data_format)
    inputs = batch_norm(inputs, training=training, data_format=data_format)
    inputs = tf.nn.leaky_relu(inputs, alpha=_LEAKY_RELU)

    return route, inputs

# 8. Detection layers

Yolo has 3 detection layers, that detect on 3 different scales using respective anchors. For each cell in the feature map, the detection layer predicts n_anchors * (5 + n_classes) values using 1x1 convolution. For each scale we have n_anchors = 3. 5 + n_classes means that respectively to each of 3 anchors we are going to predict 4 coordinates of the box, its confidence score (the probability of containing an object) and class probabilities.

In [7]:
def yolo_layer(inputs, n_classes, anchors, img_size, data_format):
    """Creates Yolo final detection layer.

    Detects boxes with respect to anchors.

    Args:
        inputs: Tensor input.
        n_classes: Number of labels.
        anchors: A list of anchor sizes.
        img_size: The input size of the model.
        data_format: The input format.

    Returns:
        Tensor output.
    """
    n_anchors = len(anchors)

    inputs = tf.layers.conv2d(inputs, filters=n_anchors * (5 + n_classes),
                              kernel_size=1, strides=1, use_bias=True,
                              data_format=data_format)

    shape = inputs.get_shape().as_list()
    grid_shape = shape[2:4] if data_format == 'channels_first' else shape[1:3]
    if data_format == 'channels_first':
        inputs = tf.transpose(inputs, [0, 2, 3, 1])
    inputs = tf.reshape(inputs, [-1, n_anchors * grid_shape[0] * grid_shape[1],
                                 5 + n_classes])

    strides = (img_size[0] // grid_shape[0], img_size[1] // grid_shape[1])

    box_centers, box_shapes, confidence, classes = \
        tf.split(inputs, [2, 2, 1, n_classes], axis=-1)

    x = tf.range(grid_shape[0], dtype=tf.float32)
    y = tf.range(grid_shape[1], dtype=tf.float32)
    x_offset, y_offset = tf.meshgrid(x, y)
    x_offset = tf.reshape(x_offset, (-1, 1))
    y_offset = tf.reshape(y_offset, (-1, 1))
    x_y_offset = tf.concat([x_offset, y_offset], axis=-1)
    x_y_offset = tf.tile(x_y_offset, [1, n_anchors])
    x_y_offset = tf.reshape(x_y_offset, [1, -1, 2])
    box_centers = tf.nn.sigmoid(box_centers)
    box_centers = (box_centers + x_y_offset) * strides

    anchors = tf.tile(anchors, [grid_shape[0] * grid_shape[1], 1])
    box_shapes = tf.exp(box_shapes) * tf.to_float(anchors)

    confidence = tf.nn.sigmoid(confidence)

    classes = tf.nn.sigmoid(classes)

    inputs = tf.concat([box_centers, box_shapes,
                        confidence, classes], axis=-1)

    return inputs

# 9. Upsample layer

In order to concatenate with shortcut outputs from Darknet-53 before applying detection on a different scale, we are going to upsample the feature map using nearest neighbor interpolation.

In [8]:
def upsample(inputs, out_shape, data_format):
    """Upsamples to `out_shape` using nearest neighbor interpolation."""
    if data_format == 'channels_first':
        inputs = tf.transpose(inputs, [0, 2, 3, 1])
        new_height = out_shape[3]
        new_width = out_shape[2]
    else:
        new_height = out_shape[2]
        new_width = out_shape[1]

    inputs = tf.image.resize_nearest_neighbor(inputs, (new_height, new_width))

    if data_format == 'channels_first':
        inputs = tf.transpose(inputs, [0, 3, 1, 2])

    return inputs

# 10. Non-max suppression

The model is going to produce a lot of boxes, so we need a way to discard the boxes with low confidence scores. Also, to avoid having multiple boxes for one object, we will discard the boxes with high overlap as well using non-max suppression for each class.

In [9]:
def build_boxes(inputs):
    """Computes top left and bottom right points of the boxes."""
    center_x, center_y, width, height, confidence, classes = \
        tf.split(inputs, [1, 1, 1, 1, 1, -1], axis=-1)

    top_left_x = center_x - width / 2
    top_left_y = center_y - height / 2
    bottom_right_x = center_x + width / 2
    bottom_right_y = center_y + height / 2

    boxes = tf.concat([top_left_x, top_left_y,
                       bottom_right_x, bottom_right_y,
                       confidence, classes], axis=-1)

    return boxes


def non_max_suppression(inputs, n_classes, max_output_size, iou_threshold,
                        confidence_threshold):
    """Performs non-max suppression separately for each class.

    Args:
        inputs: Tensor input.
        n_classes: Number of classes.
        max_output_size: Max number of boxes to be selected for each class.
        iou_threshold: Threshold for the IOU.
        confidence_threshold: Threshold for the confidence score.
    Returns:
        A list containing class-to-boxes dictionaries
            for each sample in the batch.
    """
    batch = tf.unstack(inputs)
    boxes_dicts = []
    for boxes in batch:
        boxes = tf.boolean_mask(boxes, boxes[:, 4] > confidence_threshold)
        classes = tf.argmax(boxes[:, 5:], axis=-1)
        classes = tf.expand_dims(tf.to_float(classes), axis=-1)
        boxes = tf.concat([boxes[:, :5], classes], axis=-1)

        boxes_dict = dict()
        for cls in range(n_classes):
            mask = tf.equal(boxes[:, 5], cls)
            mask_shape = mask.get_shape()
            if mask_shape.ndims != 0:
                class_boxes = tf.boolean_mask(boxes, mask)
                boxes_coords, boxes_conf_scores, _ = tf.split(class_boxes,
                                                              [4, 1, -1],
                                                              axis=-1)
                boxes_conf_scores = tf.reshape(boxes_conf_scores, [-1])
                indices = tf.image.non_max_suppression(boxes_coords,
                                                       boxes_conf_scores,
                                                       max_output_size,
                                                       iou_threshold)
                class_boxes = tf.gather(class_boxes, indices)
                boxes_dict[cls] = class_boxes[:, :5]

        boxes_dicts.append(boxes_dict)

    return boxes_dicts

# 11. Final model

Finally, let's define the model class using all of the layers described previously.

In [10]:
class Yolo_v3:
    """Yolo v3 model class."""

    def __init__(self, n_classes, model_size, max_output_size, iou_threshold,
                 confidence_threshold, data_format=None):
        """Creates the model.

        Args:
            n_classes: Number of class labels.
            model_size: The input size of the model.
            max_output_size: Max number of boxes to be selected for each class.
            iou_threshold: Threshold for the IOU.
            confidence_threshold: Threshold for the confidence score.
            data_format: The input format.

        Returns:
            None.
        """
        if not data_format:
            if tf.test.is_built_with_cuda():
                data_format = 'channels_first'
            else:
                data_format = 'channels_last'

        self.n_classes = n_classes
        self.model_size = model_size
        self.max_output_size = max_output_size
        self.iou_threshold = iou_threshold
        self.confidence_threshold = confidence_threshold
        self.data_format = data_format

    def __call__(self, inputs, training):
        """Add operations to detect boxes for a batch of input images.

        Args:
            inputs: A Tensor representing a batch of input images.
            training: A boolean, whether to use in training or inference mode.

        Returns:
            A list containing class-to-boxes dictionaries
                for each sample in the batch.
        """
        with tf.variable_scope('yolo_v3_model'):
            if self.data_format == 'channels_first':
                inputs = tf.transpose(inputs, [0, 3, 1, 2])

            inputs = inputs / 255

            route1, route2, inputs = darknet53(inputs, training=training,
                                               data_format=self.data_format)

            route, inputs = yolo_convolution_block(
                inputs, filters=512, training=training,
                data_format=self.data_format)
            detect1 = yolo_layer(inputs, n_classes=self.n_classes,
                                 anchors=_ANCHORS[6:9],
                                 img_size=self.model_size,
                                 data_format=self.data_format)

            inputs = conv2d_fixed_padding(route, filters=256, kernel_size=1,
                                          data_format=self.data_format)
            inputs = batch_norm(inputs, training=training,
                                data_format=self.data_format)
            inputs = tf.nn.leaky_relu(inputs, alpha=_LEAKY_RELU)
            upsample_size = route2.get_shape().as_list()
            inputs = upsample(inputs, out_shape=upsample_size,
                              data_format=self.data_format)
            axis = 1 if self.data_format == 'channels_first' else 3
            inputs = tf.concat([inputs, route2], axis=axis)
            route, inputs = yolo_convolution_block(
                inputs, filters=256, training=training,
                data_format=self.data_format)
            detect2 = yolo_layer(inputs, n_classes=self.n_classes,
                                 anchors=_ANCHORS[3:6],
                                 img_size=self.model_size,
                                 data_format=self.data_format)

            inputs = conv2d_fixed_padding(route, filters=128, kernel_size=1,
                                          data_format=self.data_format)
            inputs = batch_norm(inputs, training=training,
                                data_format=self.data_format)
            inputs = tf.nn.leaky_relu(inputs, alpha=_LEAKY_RELU)
            upsample_size = route1.get_shape().as_list()
            inputs = upsample(inputs, out_shape=upsample_size,
                              data_format=self.data_format)
            inputs = tf.concat([inputs, route1], axis=axis)
            route, inputs = yolo_convolution_block(
                inputs, filters=128, training=training,
                data_format=self.data_format)
            detect3 = yolo_layer(inputs, n_classes=self.n_classes,
                                 anchors=_ANCHORS[0:3],
                                 img_size=self.model_size,
                                 data_format=self.data_format)

            inputs = tf.concat([detect1, detect2, detect3], axis=1)

            inputs = build_boxes(inputs)

            boxes_dicts = non_max_suppression(
                inputs, n_classes=self.n_classes,
                max_output_size=self.max_output_size,
                iou_threshold=self.iou_threshold,
                confidence_threshold=self.confidence_threshold)

            return boxes_dicts

# 12. Utility functions

Here are some utility functions that will help us load images as NumPy arrays, load class names from the official file and draw the predicted boxes.

In [49]:
def load_images(img_names, model_size):
    """Loads images in a 4D array.

    Args:
        img_names: A list of images names.
        model_size: The input size of the model.
        data_format: A format for the array returned
            ('channels_first' or 'channels_last').

    Returns:
        A 4D NumPy array.
    """
    imgs = []

    for img_name in img_names:
        img = Image.open(img_name)
        img = img.resize(size=model_size)
        img = np.array(img, dtype=np.float32)
        img = np.expand_dims(img, axis=0)
        imgs.append(img)

    imgs = np.concatenate(imgs)

    return imgs


def load_class_names(file_name):
    """Returns a list of class names read from `file_name`."""
    with open(file_name, 'r') as f:
        class_names = f.read().splitlines()
    return class_names


def draw_boxes(img_names, boxes_dicts, class_names, model_size):
    """Draws detected boxes.

    Args:
        img_names: A list of input images names.
        boxes_dict: A class-to-boxes dictionary.
        class_names: A class names list.
        model_size: The input size of the model.

    Returns:
        None.
    """

    object_list = list()
    repeated_detected_object_list = list()
    repeated_detected_object_bounding_boxes = list()
 
    for num, img_name, boxes_dict in zip(range(len(img_names)), img_names,
                                         boxes_dicts):
        img = Image.open(img_name)
        draw = ImageDraw.Draw(img)
        font = ImageFont.truetype(font='files/futur.ttf',
                                  size=(img.size[0] + img.size[1]) // 150)
        resize_factor = \
            (img.size[0] / model_size[0], img.size[1] / model_size[1])
    ##declaration of empty lists for objects detected and bounding boxes
        repeated_detected_object = []
        detected_object = []
        cordinates_bounding_boxes = []
        for cls in range(len(class_names)):
            boxes = boxes_dict[cls]
            
            if np.size(boxes) != 0:
                color = np.random.permutation([np.random.randint(256), 255, 0])
                for box in boxes:
                    detected_object.append(class_names[cls])
        print(boxes_dict)
        cordinates_bounding_boxes.append(boxes_dict)
                        
        
        repeated_detected_object_list.append(detected_object)
        repeated_detected_object_bounding_boxes.append(cordinates_bounding_boxes)
        detected_object = list(set(detected_object))
        object_list.append(detected_object)
    return [object_list, repeated_detected_object_list, repeated_detected_object_bounding_boxes]
        

# 13. Converting weights to Tensorflow format

Now it's time to load the official weights. We are going to iterate through the file and gradually create tf.assign operations.



In [13]:
def load_weights(variables, file_name):
    """Reshapes and loads official pretrained Yolo weights.

    Args:
        variables: A list of tf.Variable to be assigned.
        file_name: A name of a file containing weights.

    Returns:
        A list of assign operations.
    """
    with open(file_name, "rb") as f:
        # Skip first 5 values containing irrelevant info
        np.fromfile(f, dtype=np.int32, count=5)
        weights = np.fromfile(f, dtype=np.float32)

        assign_ops = []
        ptr = 0

        # Load weights for Darknet part.
        # Each convolution layer has batch normalization.
        for i in range(52):
            conv_var = variables[5 * i]
            gamma, beta, mean, variance = variables[5 * i + 1:5 * i + 5]
            batch_norm_vars = [beta, gamma, mean, variance]

            for var in batch_norm_vars:
                shape = var.shape.as_list()
                num_params = np.prod(shape)
                var_weights = weights[ptr:ptr + num_params].reshape(shape)
                ptr += num_params
                assign_ops.append(tf.assign(var, var_weights))

            shape = conv_var.shape.as_list()
            num_params = np.prod(shape)
            var_weights = weights[ptr:ptr + num_params].reshape(
                (shape[3], shape[2], shape[0], shape[1]))
            var_weights = np.transpose(var_weights, (2, 3, 1, 0))
            ptr += num_params
            assign_ops.append(tf.assign(conv_var, var_weights))

        # Loading weights for Yolo part.
        # 7th, 15th and 23rd convolution layer has biases and no batch norm.
        ranges = [range(0, 6), range(6, 13), range(13, 20)]
        unnormalized = [6, 13, 20]
        for j in range(3):
            for i in ranges[j]:
                current = 52 * 5 + 5 * i + j * 2
                conv_var = variables[current]
                gamma, beta, mean, variance =  \
                    variables[current + 1:current + 5]
                batch_norm_vars = [beta, gamma, mean, variance]

                for var in batch_norm_vars:
                    shape = var.shape.as_list()
                    num_params = np.prod(shape)
                    var_weights = weights[ptr:ptr + num_params].reshape(shape)
                    ptr += num_params
                    assign_ops.append(tf.assign(var, var_weights))

                shape = conv_var.shape.as_list()
                num_params = np.prod(shape)
                var_weights = weights[ptr:ptr + num_params].reshape(
                    (shape[3], shape[2], shape[0], shape[1]))
                var_weights = np.transpose(var_weights, (2, 3, 1, 0))
                ptr += num_params
                assign_ops.append(tf.assign(conv_var, var_weights))

            bias = variables[52 * 5 + unnormalized[j] * 5 + j * 2 + 1]
            shape = bias.shape.as_list()
            num_params = np.prod(shape)
            var_weights = weights[ptr:ptr + num_params].reshape(shape)
            ptr += num_params
            assign_ops.append(tf.assign(bias, var_weights))

            conv_var = variables[52 * 5 + unnormalized[j] * 5 + j * 2]
            shape = conv_var.shape.as_list()
            num_params = np.prod(shape)
            var_weights = weights[ptr:ptr + num_params].reshape(
                (shape[3], shape[2], shape[0], shape[1]))
            var_weights = np.transpose(var_weights, (2, 3, 1, 0))
            ptr += num_params
            assign_ops.append(tf.assign(conv_var, var_weights))

    return assign_ops

# User input part

In [14]:
### path of the image
PATH = 'val2017/'

In [15]:
#### PATH + <name of image.extention>
img_names = glob.glob(PATH+'*.jpg')

In [16]:
img_names

['val2017\\000000000139.jpg',
 'val2017\\000000000285.jpg',
 'val2017\\000000000632.jpg',
 'val2017\\000000000724.jpg',
 'val2017\\000000000776.jpg',
 'val2017\\000000000785.jpg',
 'val2017\\000000000802.jpg',
 'val2017\\000000000872.jpg',
 'val2017\\000000000885.jpg',
 'val2017\\000000001000.jpg',
 'val2017\\000000001268.jpg',
 'val2017\\000000001296.jpg',
 'val2017\\000000001353.jpg',
 'val2017\\000000001425.jpg',
 'val2017\\000000001490.jpg',
 'val2017\\000000001503.jpg',
 'val2017\\000000001532.jpg',
 'val2017\\000000001584.jpg',
 'val2017\\000000001675.jpg',
 'val2017\\000000001761.jpg',
 'val2017\\000000001818.jpg',
 'val2017\\000000001993.jpg',
 'val2017\\000000002006.jpg',
 'val2017\\000000002149.jpg',
 'val2017\\000000002153.jpg',
 'val2017\\000000002157.jpg',
 'val2017\\000000002261.jpg',
 'val2017\\bike.jpg',
 'val2017\\bike2.jpg',
 'val2017\\bike3.jpg',
 'val2017\\city.jpg',
 'val2017\\handbag.jpg',
 'val2017\\mobile.jpg']

# 15. Running the model

Now we can run the model using some sample images.

Testing the model with IoU (Interception over Union ratio used in non-max suppression) threshold and confidence threshold both set to 0.5.

In [17]:
tf.reset_default_graph()

batch_size = len(img_names)
batch = load_images(img_names, model_size=_MODEL_SIZE)
#print(batch_size)
class_names = load_class_names('files/coco.names')
n_classes = len(class_names)
max_output_size = 10
iou_threshold = 0.5
confidence_threshold = 0.5

model = Yolo_v3(n_classes=n_classes, model_size=_MODEL_SIZE,
                max_output_size=max_output_size,
                iou_threshold=iou_threshold,
                confidence_threshold=confidence_threshold)

inputs = tf.placeholder(tf.float32, [batch_size, 416, 416, 3])

detections = model(inputs, training=False)

model_vars = tf.global_variables(scope='yolo_v3_model')
assign_ops = load_weights(model_vars, 'files/yolov3.weights')

with tf.Session() as sess:
    sess.run(assign_ops)
    detection_result = sess.run(detections, feed_dict={inputs: batch})

Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.BatchNormalization` documentation).
Instructions for updating:
Use `tf.cast` instead.


In [37]:
# objects_dict = dict()
# for key, values in detection_result[0].items():
#     if len(values):
#         objects_dict[class_names[key]] = len(values)
        

In [38]:
# objects = [class_names[key] for key in detection_result[0].keys()]


In [39]:
# def output(words, img_names, detection_result, class_names, _MODEL_SIZE, obj):
    
# ### Yes No type question
#     if words[0] == 'is' or words[0]=='are'  and 'locate' in final_words_lemmatizer:
#         for i in obj:
#             if i in objects_dict:
#                 print(i,':','yes')
#                 draw_boxes(img_names, detection_result, class_names, _MODEL_SIZE, obj)
#             else:
#                 print(i,':','no')
    
    
# ### Yes No type question
#     if words[0] == 'is' or words[0]=='are':
#         for i in obj:
#             if i in objects_dict:
#                 print(i,':','yes')
#             else:
#                 print(i,':','no')

# ### To count number of objects and to locate them            
#     elif words[0] == 'how'  and words[1]=='many' and ('locate' in final_words_lemmatizer or 'show' in words):
#         draw_boxes(img_names, detection_result, class_names, _MODEL_SIZE, obj)
#         for i in obj:
#             if i in objects_dict:
#                 print(i,':',objects_dict[i])

# ### To only count number of objects
#     elif words[0] == 'how'  and words[1]=='many' :
#         for i in obj:
#             if i in objects_dict:
#                 print(i,':',objects_dict[i])

# ### To only locate the objects

In [40]:
#     elif words[0] == 'where':
#         draw_boxes(img_names, detection_result, class_names, _MODEL_SIZE, obj)

#     else:
#         print('Invalid QUERY!!!')

In [41]:
# flag = False
# for i in obj:
#     if i in class_names:
#         flag = True
#         output(words, img_names, detection_result, class_names, _MODEL_SIZE, obj)
        
# if flag == False:
#     print('Sorry the object is not in the image.')

In [18]:
final_image_names =[]
for img in img_names:
    path, name = img.split('\\')
    final_image_names.append(name)

In [50]:
object_list, repeated_detected_object_list, repeated_detected_object_bounding_boxes= draw_boxes(img_names, detection_result, class_names, _MODEL_SIZE)

{0: array([[283.42548   , 164.44724   , 303.92645   , 293.98764   ,
          0.77991056],
       [268.69693   , 159.8794    , 301.80533   , 298.88007   ,
          0.70930517],
       [246.89917   , 168.93462   , 262.7517    , 209.75203   ,
          0.6187554 ]], dtype=float32), 1: array([], shape=(0, 5), dtype=float32), 2: array([], shape=(0, 5), dtype=float32), 3: array([], shape=(0, 5), dtype=float32), 4: array([], shape=(0, 5), dtype=float32), 5: array([], shape=(0, 5), dtype=float32), 6: array([], shape=(0, 5), dtype=float32), 7: array([], shape=(0, 5), dtype=float32), 8: array([], shape=(0, 5), dtype=float32), 9: array([], shape=(0, 5), dtype=float32), 10: array([], shape=(0, 5), dtype=float32), 11: array([], shape=(0, 5), dtype=float32), 12: array([], shape=(0, 5), dtype=float32), 13: array([], shape=(0, 5), dtype=float32), 14: array([], shape=(0, 5), dtype=float32), 15: array([], shape=(0, 5), dtype=float32), 16: array([], shape=(0, 5), dtype=float32), 17: array([], shape=(0,

{0: array([[340.6137    , 281.9729    , 371.4229    , 410.94055   ,
          0.98757243],
       [ 93.757195  , 280.9793    , 105.99952   , 336.00623   ,
          0.972201  ],
       [260.6676    , 279.71286   , 279.56512   , 379.43094   ,
          0.9706896 ],
       [ 22.950134  , 287.20117   ,  57.31169   , 408.5866    ,
          0.96346927],
       [ 70.65497   , 281.31354   ,  82.78104   , 334.66602   ,
          0.94204223],
       [366.0957    , 276.6803    , 377.62415   , 340.813     ,
          0.9214185 ],
       [109.59676   , 280.85016   , 119.73188   , 337.68378   ,
          0.9032631 ],
       [ 61.331482  , 282.03638   ,  70.913315  , 334.05426   ,
          0.80960476],
       [380.11575   , 273.6099    , 391.26968   , 344.2225    ,
          0.7938415 ],
       [ 86.33277   , 278.60303   ,  94.538795  , 334.64697   ,
          0.6544842 ]], dtype=float32), 1: array([], shape=(0, 5), dtype=float32), 2: array([[134.03494   , 278.00552   , 169.81714   , 345.24826   ,

In [45]:
repeated_detected_object_bounding_boxes[31]

[{0: array([[318.7868    ,   7.9528046 , 400.81824   , 409.3216    ,
            0.9994122 ],
         [  8.489849  ,   5.0512695 , 104.90184   , 425.19547   ,
            0.99886   ],
         [121.670654  ,  -8.707901  , 194.44604   , 426.65765   ,
            0.9972728 ],
         [223.83786   ,   8.065353  , 308.207     , 409.08838   ,
            0.98784435]], dtype=float32),
  1: array([], shape=(0, 5), dtype=float32),
  2: array([], shape=(0, 5), dtype=float32),
  3: array([], shape=(0, 5), dtype=float32),
  4: array([], shape=(0, 5), dtype=float32),
  5: array([], shape=(0, 5), dtype=float32),
  6: array([], shape=(0, 5), dtype=float32),
  7: array([], shape=(0, 5), dtype=float32),
  8: array([], shape=(0, 5), dtype=float32),
  9: array([], shape=(0, 5), dtype=float32),
  10: array([], shape=(0, 5), dtype=float32),
  11: array([], shape=(0, 5), dtype=float32),
  12: array([], shape=(0, 5), dtype=float32),
  13: array([], shape=(0, 5), dtype=float32),
  14: array([], shape=(0, 5

In [21]:
category = pd.read_csv('tagged_data/category.csv')
category

,class_name,category_name
0,person,person
1,bicycle,transport
2,car,transport
3,motorbike,transport
4,aeroplane,transport
...,...,...
75,vase,household
76,scissors,household
77,teddy bear,other
78,hair drier,household


In [39]:
category_list = []

for obj in object_list:
    category_list.append([category[category.class_name == class_name].category_name.values[0] for class_name in obj])
    
category_list = [list(set(classes)) for classes in category_list]
category_list

[['person', 'gadget', 'furniture', 'household'],
 ['animal'],
 ['gadget', 'furniture', 'other', 'household'],
 ['transport'],
 ['other'],
 ['person', 'sports'],
 ['household'],
 ['person', 'sports'],
 ['person', 'sports'],
 ['person', 'sports', 'wearable'],
 ['person', 'wearable'],
 ['person', 'gadget'],
 ['person'],
 ['food', 'household'],
 ['person', 'sports'],
 ['gadget'],
 ['transport'],
 ['person', 'transport'],
 ['animal', 'wearable'],
 ['transport'],
 ['animal'],
 ['furniture'],
 ['person', 'transport'],
 ['food', 'household'],
 ['person', 'sports'],
 ['food', 'furniture', 'household'],
 ['person', 'sports'],
 ['person', 'transport'],
 ['person', 'transport', 'wearable'],
 ['person', 'transport'],
 ['person', 'transport', 'wearable'],
 ['person', 'wearable'],
 ['person', 'gadget', 'other']]

In [41]:
file = pd.DataFrame(columns=['image_name', 'objects','category','repeated_objects','repeated_objects_coordinates','model_size'])
file['objects'] = file['objects'].astype('object')
file['category'] = file['category'].astype('object')
file['image_name'] = final_image_names
file['objects'] = object_list
file['category'] = category_list
file['repeated_objects'] = repeated_detected_object_list
file['repeated_objects_coordinates'] = repeated_detected_object_bounding_boxes
file.iloc[0]['model_size'] = _MODEL_SIZE
file

,image_name,objects,category,repeated_objects,repeated_objects_coordinates,model_size
0,000000000139.jpg,"[person, vase, chair, tvmonitor]","[person, gadget, furniture, household]","[person, person, person, chair, chair, tvmonit...",[{0: [[283.42548 164.44724 303.92645 ...,"(416, 416)"
1,000000000285.jpg,[bear],[animal],[bear],"[{0: [], 1: [], 2: [], 3: [], 4: [], 5: [], 6:...",NaN
2,000000000632.jpg,"[book, bottle, chair, bed, pottedplant, tvmoni...","[gadget, furniture, other, household]","[bottle, chair, pottedplant, pottedplant, bed,...","[{0: [], 1: [], 2: [], 3: [], 4: [], 5: [], 6:...",NaN
3,000000000724.jpg,[stop sign],[transport],"[stop sign, stop sign]","[{0: [], 1: [], 2: [], 3: [], 4: [], 5: [], 6:...",NaN
4,000000000776.jpg,[teddy bear],[other],"[teddy bear, teddy bear]","[{0: [], 1: [], 2: [], 3: [], 4: [], 5: [], 6:...",NaN
5,000000000785.jpg,"[person, skis]","[person, sports]","[person, skis]",[{0: [[184.56575 45.949722 318.38983 ...,NaN
6,000000000802.jpg,"[oven, refrigerator]",[household],"[oven, refrigerator]","[{0: [], 1: [], 2: [], 3: [], 4: [], 5: [], 6:...",NaN
7,000000000872.jpg,"[person, baseball glove]","[person, sports]","[person, person, baseball glove]",[{0: [[104.84656 74.73595 252.87292 ...,NaN
8,000000000885.jpg,"[person, tennis racket]","[person, sports]","[person, person, person, person, person, perso...",[{0: [[179.9573 174.67824 276.92285 ...,NaN
9,000000001000.jpg,"[person, handbag, tennis racket]","[person, sports, wearable]","[person, person, person, person, person, perso...",[{0: [[172.15987 92.2438 230.15309 342...,NaN


In [42]:
file.to_pickle("tagged_data/tag_data.pkl")
print("completed")

completed


In [ ]:
categorical_image = pd.DataFrame({'category':file.category.sum(), 'image':file.image_name.repeat(file.category.str.len())})
categorical_image.to_pickle("tagged_data/category_wise_image.pkl")

In [ ]:
categorical_image